# BM25 Document Search and Evaluation

This Information Retrieval System allows users to index a corpus of documents and perform searches using a BM25 ranking model. The system is designed for processing large datasets efficiently by utilizing a SPIMI (Single Pass In-Memory Indexing) indexing technique.

## Features

- **Tokenizer**: Supports tokenization with options for case normalization, stopword removal, and stemming.
- **Indexing**: Indexes documents in batches, allowing for scalability.
- **Searching**: Implements the BM25 ranking model for retrieving relevant documents based on queries.

In [ ]:
import sys
import os
import time
import ujson
from typing import List, Dict
from pathlib import Path

# Add parent directory to path to import from src
sys.path.append('..')

# Import our custom modules
import src.data_processing as dp
import src.bm25_search as bm25
import src.evaluation as ndcg 

## Configuration

The tokenizer processes text into tokens suitable for indexing and searching. It includes the following options:

- **Case Normalization**: Converts all tokens to lowercase if `lowercase=True`.
- **Minimum Token Length**: Discards tokens shorter than `min_token_length` (default is 3).
- **Stopword Removal**: Removes common stopwords if a stopword list is provided.
- **Stemming**: Reduces tokens to their stem forms using the Snowball Stemmer if `stem=True`.

In [ ]:
# Paths configuration
CORPUS_PATH = '../data/MEDLINE_2024_Baseline.jsonl'
OUTPUT_DIR = '../output'
QUESTIONS_PATH = '../data/questions.jsonl'  # Optional: for batch processing
INDEX_DIR = OUTPUT_DIR + '/index'

# Check if input files exist
if not Path(CORPUS_PATH).exists():
    print(f"Corpus file not found: {CORPUS_PATH}")
if not Path(QUESTIONS_PATH).exists():
    print(f"Questions file not found: {QUESTIONS_PATH}")

# Create output directory if it doesn't exist
Path(INDEX_DIR).mkdir(parents=True, exist_ok=True)
print(f"Output directory ready: {INDEX_DIR}")

# Tokenizer configuration
TOKENIZER_CONFIG = {
    'min_token_length': 3,
    'lowercase': True,
    'stem': True,
    'stopwords': None  # Can provide a set of stopwords
}

# BM25 parameters
BM25_PARAMS = {
    'k1': 1.2,  # Term frequency saturation
    'b': 0.75   # Document length normalization
}

# Indexing parameters
BATCH_SIZE = 10000
N_RESULTS = 100


## Corpus Indexing

Run this section to build or select a existing index file. 

### Optimization Techniques in Indexing

- **SPIMI Algorithm**: Implements the Single-Pass In-Memory Indexing algorithm to efficiently handle large datasets by writing partial indexes to disk.
- **Batch Processing**: Indexes documents in batches (`batch_size=10000`) to manage memory usage.
- **Precompiled Regular Expressions**: Uses precompiled regex patterns in the tokenizer to improve tokenization speed.
- **Stemming Cache**: Caches stemmed tokens to avoid redundant computations during tokenization.
- **MessagePack Serialization**: Uses `MessagePack` for efficient binary serialization when writing partial and merged indexes to disk.

### Index File Format
- **Format**: The index is stored as a MessagePack (`.msgpack`) file.

- **Structure**:
  - **Index**: A dictionary where keys are terms and values are dictionaries mapping document IDs to lists of positions where the term occurs.
  - **Document Lengths**: A dictionary mapping document IDs to the total number of tokens in each document.

The (`.msgpack`) file is a binary format, making it impossible to provide a screenshot of its contents. 
However, a sample from the file is shown below:

```json
{
  "index": {
    "ethylhexyl": {
      "25153068": [7, 49],
      "32745991": [3, 26],
      "12437285": [8, 10],
      "15924484": [5, 27],
      "19555962": [150],
      "12270607": [8, 29],
      "23356645": [3, 22],
      "22041199": [9, 17],
      "8333024": [3, 25],
      "20453712": [14, 22],
      "30960727": [97],
      "37536456": [13],
      "14687758": [99],
      "35843048": [21],
      "16956469": [22],
      "34788783": [1, 39, 50, 52, 83, 117, 265, 281],
      "14998748": [14, 17, 28],
      "32610232": [0, 48],
      "14556481": [32],
      "35859238": [35],
      "28661659": [84, 92],
      "31033968": [12],
        .
        .
        .
    },
        .
        .
        .
  },
  "doc_lengths": {
    "2451706": 115,
    "35308048": 192,
    "7660250": 51,
    "28963802": 143,
    "25153068": 231,
    "874026": 101,
    "4001859": 137,
    "10149271": 92,
    "35267334": 190,
    "3656477": 128,
    "30818862": 217,
        .
        .
        .
  }
}
```

### Build Index

In [ ]:
print(f"Building index from corpus: {CORPUS_PATH}")
print(f"Output directory: {INDEX_DIR}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Tokenizer config: {TOKENIZER_CONFIG}")
    
start_time = time.time()

# Index documents
index_path = dp.index_documents(
    corpus_path=CORPUS_PATH,
    output_dir=INDEX_DIR,
    batch_size=BATCH_SIZE,
    tokenizer_config=TOKENIZER_CONFIG
)

elapsed_time = time.time() - start_time
print(f"\nIndexing completed in {elapsed_time:.2f} seconds")
print(f"Index saved at: {index_path}")

### Set Existing Index Path (if there's a `merged_index.msgpack` file already)

In [ ]:
index_path = os.path.join(INDEX_DIR, 'merged_index.msgpack')
print(f"Using existing index: {index_path}")

## Load tokenizer configuration

In [ ]:
# Load tokenizer configuration
print("Tokenizer configuration:")
for key, value in TOKENIZER_CONFIG.items():
    print(f"  {key}: {value}")

## Batch Query Processing

Process multiple queries from a file.

In [ ]:
# Load queries from file
def load_queries(file_path: str) -> List[Dict]:
    """Load queries from a JSONL file."""
    queries = []
    with open(file_path, 'r') as f:
        for line in f:
            queries.append(ujson.loads(line))
    return queries

# Process batch queries if file exists
if os.path.exists(QUESTIONS_PATH):
    print(f"Loading queries from: {QUESTIONS_PATH}")
    queries = load_queries(QUESTIONS_PATH)
    print(f"Loaded {len(queries)} queries")
    
    # Process all queries
    start_time = time.time()
    batch_results = bm25.batch_search(
        queries=queries,
        index_path=index_path,
        tokenizer_config=TOKENIZER_CONFIG,
        n_results=N_RESULTS,
        k1=BM25_PARAMS['k1'],
        b=BM25_PARAMS['b']
    )
    batch_time = time.time() - start_time
    
    print(f"\nProcessed {len(batch_results)} queries in {batch_time:.2f} seconds")
    print(f"Average time per query: {batch_time/len(batch_results):.3f} seconds")
    
    # Save results
    output_file = os.path.join(OUTPUT_DIR, 'ranked_questions.jsonl')
    with open(output_file, 'w') as f:
        for entry in batch_results:
            f.write(ujson.dumps(entry) + '\n')
    
    print(f"\nResults saved to: {output_file}")
else:
    print(f"Questions file not found at: {QUESTIONS_PATH}")

## Evaluate Retrieved Documents

Here we compute the **Normalized Discounted Cumulative Gain (nDCG)** metric, which evaluates the quality of the ranked retrieval results.

### How nDCG Works

- **DCG (Discounted Cumulative Gain)**: Measures the gain (relevance) of each document in the result list, discounted by its position in the list.
- **IDCG (Ideal DCG)**: The maximum possible DCG achievable, obtained by an ideal ranking of documents.
- **nDCG**: The ratio of DCG to IDCG, normalized to a value between 0 and 1.

In [ ]:
# Compute nDCG for the given results
ndcg.compute_average_ndcg(
    questions_file_path=QUESTIONS_PATH,
    results_file_path=output_file,
    k=10
    )